In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_period', 'koi_fpflag_co', 'koi_prad', 'koi_steff', 'koi_duration', 
                        'koi_time0bk', 'koi_kepmag', 'koi_model_snr', 'koi_duration_err2', 'koi_duration_err1', 'koi_prad_err1', 'koi_steff_err1', 
                        'koi_fpflag_ec', 'koi_prad_err2', 'koi_steff_err2', 'koi_time0bk_err1', 'koi_depth', 'koi_time0bk_err2', 'koi_period_err2', 
                        'koi_insol_err1', 'koi_impact', 'koi_period_err1', 'koi_insol_err2', 'koi_insol']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
X = selected_features.drop("koi_disposition", axis=1)
y = selected_features[['koi_disposition']]
print(X.shape, y.shape)

(6991, 26) (6991, 1)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_period,koi_fpflag_co,koi_prad,koi_steff,koi_duration,koi_time0bk,koi_kepmag,koi_model_snr,...,koi_steff_err2,koi_time0bk_err1,koi_depth,koi_time0bk_err2,koi_period_err2,koi_insol_err1,koi_impact,koi_period_err1,koi_insol_err2,koi_insol
6122,0,0,6.768901,0,1.24,5737,3.61600,133.077240,14.725,10.8,...,-171,0.008440,123.1,-0.008440,-7.380000e-05,204.89,0.150,7.380000e-05,-103.87,253.30
6370,0,1,0.733726,0,0.86,5855,2.30900,132.020050,15.770,13.8,...,-175,0.007950,114.6,-0.007950,-6.060000e-06,2253.61,0.291,6.060000e-06,-677.78,2891.64
2879,1,0,7.652707,0,3.21,6328,79.89690,134.460380,13.099,254.3,...,-189,0.006190,641.1,-0.006190,-6.540000e-05,195.16,0.970,6.540000e-05,-64.34,226.81
107,0,0,7.953547,0,2.25,4768,2.63120,174.662240,15.660,38.4,...,-85,0.001820,875.4,-0.001820,-1.910000e-05,7.15,0.300,1.910000e-05,-10.12,55.37
29,0,0,4.959319,0,12.21,5712,2.22739,172.258529,15.263,696.5,...,-77,0.000083,9802.0,-0.000083,-5.150000e-07,146.52,0.831,5.150000e-07,-93.21,349.40


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


# Scale your data
X_scaler = MinMaxScaler().fit(X)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Train the Model



In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=120, activation='relu', input_dim=26))
model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 120)               3240      
_________________________________________________________________
dense_1 (Dense)              (None, 120)               14520     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 363       
Total params: 18,123
Trainable params: 18,123
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=250,
    shuffle=True,
    verbose=2
)

Epoch 1/250
164/164 - 0s - loss: 0.5393 - accuracy: 0.7149
Epoch 2/250
164/164 - 0s - loss: 0.3653 - accuracy: 0.8104
Epoch 3/250
164/164 - 0s - loss: 0.3512 - accuracy: 0.8224
Epoch 4/250
164/164 - 0s - loss: 0.3407 - accuracy: 0.8259
Epoch 5/250
164/164 - 0s - loss: 0.3323 - accuracy: 0.8407
Epoch 6/250
164/164 - 0s - loss: 0.3265 - accuracy: 0.8446
Epoch 7/250
164/164 - 0s - loss: 0.3183 - accuracy: 0.8522
Epoch 8/250
164/164 - 0s - loss: 0.3132 - accuracy: 0.8541
Epoch 9/250
164/164 - 0s - loss: 0.3143 - accuracy: 0.8566
Epoch 10/250
164/164 - 0s - loss: 0.3108 - accuracy: 0.8632
Epoch 11/250
164/164 - 0s - loss: 0.3091 - accuracy: 0.8533
Epoch 12/250
164/164 - 0s - loss: 0.3005 - accuracy: 0.8697
Epoch 13/250
164/164 - 0s - loss: 0.3037 - accuracy: 0.8665
Epoch 14/250
164/164 - 0s - loss: 0.2947 - accuracy: 0.8716
Epoch 15/250
164/164 - 0s - loss: 0.3034 - accuracy: 0.8642
Epoch 16/250
164/164 - 0s - loss: 0.2922 - accuracy: 0.8716
Epoch 17/250
164/164 - 0s - loss: 0.2933 - accura

In [14]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2905 - accuracy: 0.8965
Loss: 0.29049086570739746, Accuracy: 0.8964530825614929


# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

# from sklearn.externals import joblib
# filename = 'TylerNguyen.sav'
# joblib.dump(model, filename)

model.save('model/TylerNguyen.hdf5')